# Artificial Neural Network: NBA Player Dataset Team Optimazitation

## Team

Gabriel Aracena
Joshua Canode
Aaron Galicia

## Abstract

The objective is to use a deep artificial neural network (ANN) to determine an optimal team composition from a pool of basketball players. Given player characteristics, we want to identify the best five players that result in a balanced team.

### Data Preparation:

* Load the NBA Players Dataset.
* Filter to get a pool of 100 players from a random 5-year window.
* Normalize/Standardize player characteristics.

### ANN Model Building:

* Design a Multi-layer Perceptron (MLP) based on the architecture of the CST-435 An Artificial Neural Network Model Image (see below)
* Define layers: Input layer, Hidden layers, and Output layer.
* Determine the appropriate activation function, optimizer, and loss function for the MLP.

![ANNModel](ANNModel.png)

### Training the ANN:

* Forward propagation: Use player characteristics to propagate input data through the network and generate an output.
* Calculate the error using a predefined cost function.
* Backpropagate the error to update model weights.
* Repeat the above steps for several epochs.

### Evaluation and Team Selection:

* Use forward propagation on the trained ANN to predict player effectiveness or class labels.
* Apply a threshold function to these predictions.
* Select the top five players that meet the optimal team criteria.

## Model Architecture

* Input Layer: This layer will have neurons equal to the number of player characteristics we're considering (e.g. points, assists, offensive rebounds, defensive rebounds,etc.).
* Hidden Layers: Multiple hidden layers can be used to capture intricate patterns and relationships. We are going to have 5 hidden layers, each one of them will test the match for the players 
* Output Layer: This layer can have neurons equal to the number of classes or roles in the team we're predicting for (e.g., point guard, shooting guard, center, etc.). Each neuron will give the likelihood of a player fitting that role.

## Activation and Threshold Function

During forward propagation, each neuron processes input data and transmits it to the next layer. An activation function is applied to this data. For this model, we can use the ReLU (Rectified Linear Unit) activation function for hidden layers due to its computational efficiency and the ability to handle non-linearities. The softmax function might be applied to the output layer as it provides a probability distribution.

After obtaining the output, a threshold function is applied to convert continuous values into distinct class labels. In this case, it can be the player's most likely role in the team.

## Interpretation and Conclusion

The final output provides us with a categorization of each player in our pool. By examining the predicted class labels and the associated probabilities, we can:
* Identify which role or position each player is most suited for.
* Select the top players for each role to form our optimal team.

It's worth noting that the "optimal" team is contingent on the data provided and the neural network's training. For better results, the model should be regularly trained with updated data, and other external factors (like team chemistry and current form) should also be considered in real-world scenarios. For our optimal team we defined some weights based on each player position that will take into account the 2 most important stats for each position according to our criteria. See Definig player types bellow:


## Defining Player types    

In [1]:
"""
5 center
	height = 0.5
	weight = 0.5

4 forward
	net_rating = 0.6
	reb = 0.4

3 small forward
	ast_pct = 0.3
	usg_pct = 0.7

2 guard
	pts = 0.8
	ts_pct = 0.2

1 point guard
	ast = 0.8
	gp = 0.2


"""

'\n5 center\n\theight = 0.5\n\tweight = 0.5\n\n4 forward\n\tnet_rating = 0.6\n\treb = 0.4\n\n3 small forward\n\tast_pct = 0.3\n\tusg_pct = 0.7\n\n2 guard\n\tpts = 0.8\n\tts_pct = 0.2\n\n1 point guard\n\tast = 0.8\n\tgp = 0.2\n\n\n'

In [2]:
import pandas as pd
import random


In [3]:

# Specify the file path
file_path = "all_seasons.csv"

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(file_path)

# Display the head (first few rows) of the DataFrame
print("Head of the DataFrame:")
print(df.head())

# Display the tail (last few rows) of the DataFrame
print("\nTail of the DataFrame:")
print(df.tail())


Head of the DataFrame:
   Unnamed: 0        player_name team_abbreviation   age  player_height  \
0           0      Dennis Rodman               CHI  36.0         198.12   
1           1  Dwayne Schintzius               LAC  28.0         215.90   
2           2       Earl Cureton               TOR  39.0         205.74   
3           3        Ed O'Bannon               DAL  24.0         203.20   
4           4        Ed Pinckney               MIA  34.0         205.74   

   player_weight                      college country draft_year draft_round  \
0      99.790240  Southeastern Oklahoma State     USA       1986           2   
1     117.933920                      Florida     USA       1990           1   
2      95.254320                Detroit Mercy     USA       1979           3   
3     100.697424                         UCLA     USA       1995           1   
4     108.862080                    Villanova     USA       1985           1   

   ...  pts   reb  ast  net_rating  oreb_pct 

In [4]:
# Defining target stats based on player types

PROPORTINALITY_FACTOR = 1000

def calculateTargetValue(position, stat1, stat2):
    # Point Guard: ast = 0.8 gp = 0.2
    # Shooting Guard: pts = 0.8 ts_pct = 0.2
    if (position == 1 or position == 2):
        weightedValue = ((stat1 + PROPORTINALITY_FACTOR) * 0.8 + (stat2 + PROPORTINALITY_FACTOR) * 0.2 ) / PROPORTINALITY_FACTOR
        return weightedValue
    
    # Small Forward: ast_pct = 0.3 usg_pct = 0.7
    elif (position == 3):
        weightedValue = ((stat1 + PROPORTINALITY_FACTOR) * 0.3 + (stat2 + PROPORTINALITY_FACTOR) * 0.7 ) / PROPORTINALITY_FACTOR
        return weightedValue
    # Forward: net_rating = 0.6 reb = 0.4
    elif (position == 4):
        weightedValue = ((stat1 + PROPORTINALITY_FACTOR) * 0.6 + (stat2 + PROPORTINALITY_FACTOR) * 0.4 ) / PROPORTINALITY_FACTOR
        return weightedValue
    # Center: height = 0.5 weight = 0.5
    elif (position == 5):
        weightedValue = ((stat1 + PROPORTINALITY_FACTOR) * 0.6 + (stat2 + PROPORTINALITY_FACTOR) * 0.4 ) / PROPORTINALITY_FACTOR
        return weightedValue

MAXIMUM_ASSIST = max(df['ast'])
MAXIMUM_GP = max(df['gp'])
MAXIMUM_PTS = max(df['pts'])
MAXIMUM_SHOOTING_RATE = max(df['ts_pct'])
MAXIMUM_ASSIST_PCTG = max(df['ast_pct'])
MAXIMUM_USG_PCT = max(df['usg_pct']) 
MAXIMUM_NET_RATING = max(df['pts'])
MAXIMUM_REB = max(df['oreb_pct'])
MAXIMUM_HEIGHT = max(df['player_height']) 
MAXIMUM_WEIGHT = max(df['player_weight']) 

# The target stats will be 80% of the maximum value (it will be really hard to get 100% all the time since we are going to only use 100 players out of the whole dataset)
TARGET_POINT_GUARD_VALUE = calculateTargetValue(1, MAXIMUM_ASSIST, MAXIMUM_GP)
TARGET_SHOOTING_GUARD_VALUE = calculateTargetValue(2, MAXIMUM_PTS, MAXIMUM_SHOOTING_RATE)
TARGET_SMALL_FORWARD_VALUE = calculateTargetValue(3, MAXIMUM_ASSIST_PCTG, MAXIMUM_USG_PCT)
TARGET_FORWARD_VALUE = calculateTargetValue(4, MAXIMUM_NET_RATING, MAXIMUM_REB)
TARGET_CENTER_VALUE = calculateTargetValue(5, MAXIMUM_HEIGHT, MAXIMUM_WEIGHT)

print(TARGET_POINT_GUARD_VALUE)
print(TARGET_SHOOTING_GUARD_VALUE)
print(TARGET_SMALL_FORWARD_VALUE)
print(TARGET_FORWARD_VALUE)
print(TARGET_CENTER_VALUE)



1.0263600000000002
1.02918
1.001
1.02206
1.204001248


In [8]:
df['draft_year'] = df['season'].str.split('-').str[0].astype(int)

# Define the target year and the window size
target_year = 1996  # Replace with your desired target year
window_size = 5

# Calculate the start and end years of the window
start_year = target_year - window_size
end_year = target_year

# Filter the dataset to include only records within the 5-year window
filtered_df = df[(df['draft_year'] >= start_year) & (df['draft_year'] <= end_year)]

# Ensure the filtered dataset has at least 100 players
if len(filtered_df) < 100:
    print("There are not enough players within the specified window.")
else:
    # Randomly select 100 players from the filtered dataset
    random.seed(42)  # Set a random seed for reproducibility
    selected_players = random.sample(range(len(filtered_df)), 100)

    # Create a DataFrame containing the selected players
    selected_df = filtered_df.iloc[selected_players]

    # Display the selected players
    print(selected_df)


     Unnamed: 0       player_name team_abbreviation   age  player_height  \
327         327     Tom Gugliotta               MIN  27.0         208.28   
57           57  Hot Rod Williams               PHX  34.0         210.82   
12           12     Emanual Davis               HOU  28.0         195.58   
379         379      Jud Buechler               CHI  29.0         198.12   
140         140    Aaron Williams               VAN  25.0         205.74   
..          ...               ...               ...   ...            ...   
435         435      Martin Lewis               TOR  22.0         198.12   
358         358       Kenny Smith               DEN  32.0         190.50   
236         236    Sedale Threatt               HOU  35.0         187.96   
363         363     Jimmy Carruth               MIL  27.0         208.28   
138         138        A.C. Green               DAL  33.0         205.74   

     player_weight                          college country  draft_year  \
327     108.

In [9]:


import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split

# Split the selected players into training (90%) and validation (10%) sets
train_df, val_df = train_test_split(selected_df, test_size=0.1, random_state=42)


# Define your model architecture
model = models.Sequential([
    # ... Define layers ...
])

# Compile the model with appropriate loss and optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Perform training with train_df and validate on val_df
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))




NameError: name 'models' is not defined

In [ ]:
#TODO

# Load the remaining players dataset (unseen data)

# Preprocess the unseen data (normalize/standardize if needed)
# ...

# Use the trained model for role prediction on unseen data
predictions = model.predict(unseen_data)
